# python logging模块使用教程
原文：https://www.jianshu.com/p/feb86c06c4f4

In [3]:
import logging

logging.debug("debug message")
logging.info("info message")
logging.warning("warn message")
logging.error("error message")
logging.critical("critical message")

ERROR:root:error message
CRITICAL:root:critical message


默认情况下，logging模块将日志打印到屏幕上(stdout)，日志级别为WARNING(即只有日志级别高于WARNING的日志信息才会输出)，日志格式如下图所示：
![image](Images/2019_3_27_1.jpg)

## 简单配置
### 日志级别
级别|何时使用
--|:--:
DEBUG|详细信息，典型地调试问题时会感兴趣。
INFO|证明事情按预期工作
WARNING|表明发生了一些意外，或者不久的将来会发生问题（如‘磁盘满了’）。软件还是在正常工作。
ERROR|由于更严重的问题，软件已不能执行一些功能了。
CRITICAL|严重错误，表明软件已不能继续运行了。

## 配置

In [7]:
import logging

logging.basicConfig(filename = 'logger.log',level=logging.INFO)

logging.debug("debug message")
logging.info("info message")
logging.warning("warn message")
logging.error("error message")
logging.critical("critical message")

ERROR:root:error message
CRITICAL:root:critical message


因为通过level=logging.INFO设置日志级别为INFO，所以所有的日志信息均输出出来了。

# 几个重要的概念
 * Logger 记录器，暴露了应用程序代码能直接使用的接口。
 * Handler 处理器，将（记录器产生的）日志记录发送至合适的目的地。
 * Filter 过滤器，提供了更好的粒度控制，它可以决定输出哪些日志记录。
 * Formatter 格式化器，指明了最终输出中日志记录的布局。

## Logger 记录器
Logger是一个树形层级结构，在使用接口debug，info，warn，error，critical之前必须创建Logger实例，即创建一个记录器，如果没有显式的进行创建，则默认创建一个root logger，并应用默认的日志级别(WARN)，处理器Handler(StreamHandler，即将日志信息打印输出在标准输出上)，和格式化器Formatter(默认的格式即为第一个简单使用程序中输出的格式)。

```创建方法: logger = logging.getLogger(logger_name)```

In [11]:
logger = logging.getLogger("my_logger")
logger.setLevel(logging.ERROR) # 设置日志级别为ERROR，即只有日志级别大于等于ERROR的日志才会输出
logger.addHandler("my_handler") # 为Logger实例增加一个处理器
logger.removeHandler("my_handler") # 为Logger实例删除一个处理器

## Handler 处理器
Handler处理器类型有很多种，比较常用的有三个，StreamHandler，FileHandler，NullHandler，详情可以访问Python logging.handlers
创建StreamHandler之后，可以通过使用以下方法设置日志级别，设置格式化器Formatter，增加或删除过滤器Filter。

In [17]:
handler = logging.Handler()

handler.setLevel(logging.WARN) # 指定日志级别，低于WARN级别的日志将被忽略
handler.setFormatter("formatter_name") # 设置一个格式化器formatter
handler.addFilter("filter_name") # 增加一个过滤器，可以增加多个
handler.removeFilter("filter_name") # 删除一个过滤器

### StreamHandler

In [18]:
sh = logging.StreamHandler(stream=None)

### FileHandler

In [19]:
fh = logging.FileHandler(filename, mode='a', encoding=None, delay=False)

NameError: name 'filename' is not defined

## Formatter 格式化器
使用Formatter对象设置日志信息最后的规则、结构和内容，默认的时间格式为%Y-%m-%d %H:%M:%S。

In [20]:
formatter = logging.Formatter(fmt=None, datefmt=None)

## Filter 过滤器
Handlers和Loggers可以使用Filters来完成比级别更复杂的过滤。Filter基类只允许特定Logger层次以下的事件。例如用‘A.B’初始化的Filter允许Logger ‘A.B’, ‘A.B.C’, ‘A.B.C.D’, ‘A.B.D’等记录的事件，logger‘A.BB’, ‘B.A.B’ 等就不行。 如果用空字符串来初始化，所有的事件都接受。

In [21]:
filter = logging.Filter(name='')

![image](Images/2019_3_27_2.jpg)

# Logging工作流程
logging模块使用过程

第一次导入logging模块或使用reload函数重新导入logging模块，logging模块中的代码将被执行，这个过程中将产生logging日志系统的默认配置。
自定义配置(可选)。logging标准模块支持三种配置方式: dictConfig，fileConfig，listen。其中，dictConfig是通过一个字典进行配置Logger，Handler，Filter，Formatter；fileConfig则是通过一个文件进行配置；而listen则监听一个网络端口，通过接收网络数据来进行配置。当然，除了以上集体化配置外，也可以直接调用Logger，Handler等对象中的方法在代码中来显式配置。
使用logging模块的全局作用域中的getLogger函数来得到一个Logger对象实例(其参数即是一个字符串，表示Logger对象实例的名字，即通过该名字来得到相应的Logger对象实例)。
使用Logger对象中的debug，info，error，warn，critical等方法记录日志信息。

logging模块处理流程
![images](Images/2019_3_27_3.jpg)
  * 判断日志的等级是否大于Logger对象的等级，如果大于，则往下执行，否则，流程结束。
  * 产生日志。第一步，判断是否有异常，如果有，则添加异常信息。第二步，处理日志记录方法(如debug，info等)中的占位符，即一般的字符串格式化处理。
  * 使用注册到Logger对象中的Filters进行过滤。如果有多个过滤器，则依次过滤；只要有一个过滤器返回假，则过滤结束，且该日志信息将丢弃，不再处理，而处理流程也至此结束。否则，处理流程往下执行。
  * 在当前Logger对象中查找Handlers，如果找不到任何Handler，则往上到该Logger对象的父Logger中查找；如果找到一个或多个Handler，则依次用Handler来处理日志信息。但在每个Handler处理日志信息过程中，会首先判断日志信息的等级是否大于该Handler的等级，如果大于，则往下执行(由Logger对象进入Handler对象中)，否则，处理流程结束。
  * 执行Handler对象中的filter方法，该方法会依次执行注册到该Handler对象中的Filter。如果有一个Filter判断该日志信息为假，则此后的所有Filter都不再执行，而直接将该日志信息丢弃，处理流程结束。
  * 使用Formatter类格式化最终的输出结果。 注：Formatter同上述第2步的字符串格式化不同，它会添加额外的信息，比如日志产生的时间，产生日志的源代码所在的源文件的路径等等。
  * 真正地输出日志信息(到网络，文件，终端，邮件等)。至于输出到哪个目的地，由Handler的种类来决定。

## 再看日志配置
### 配置方式

 * 显式创建记录器Logger、处理器Handler和格式化器Formatter，并进行相关设置；
 * 通过简单方式进行配置，使用basicConfig()函数直接进行配置；
 * 通过配置文件进行配置，使用fileConfig()函数读取配置文件；
 * 通过配置字典进行配置，使用dictConfig()函数读取配置信息；
 * 通过网络进行配置，使用listen()函数进行网络配置。

### basicConfig关键字参数

关键字|描述
--|:--:
filename|创建一个FileHandler，使用指定的文件名，而不是使用StreamHandler。
filemode|如果指明了文件名，指明打开文件的模式（如果没有指明filemode，默认为'a'）。
format|handler使用指明的格式化字符串。
datefmt|使用指明的日期／时间格式。
level|指明根logger的级别。
stream|使用指明的流来初始化StreamHandler。该参数与'filename'不兼容，如果两个都有，'stream'被忽略。

有用的format格式


格式|描述
--|:--:
%(levelno)s|打印日志级别的数值
%(levelname)s|打印日志级别名称
%(pathname)s|打印当前执行程序的路径
%(filename)s|打印当前执行程序名称
%(funcName)s|打印日志的当前函数
%(lineno)s|打印日志的当前行号
%(asctime)s|打印日志的时间
%(thread)s|打印线程id
%(threadName)s|打印线程名称
%(process)s|打印进程ID
%(message)s|打印日志信息